In [ ]:
import torch
from insightface.model_zoo import get_model
from torch import nn
from torch.utils.data import DataLoader
from insightface.data import get_dataset
from insightface.utils import L2Norm


In [ ]:
import torch
import torch.nn.functional as F

class ArcFaceLoss(nn.Module):
    def __init__(self, s=64.0, m=0.50, easy_margin=False):
        super(ArcFaceLoss, self).__init__()
        self.s = s
        self.m = m
        self.easy_margin = easy_margin

    def forward(self, embedding, label):
        cos_theta = F.linear(F.normalize(embedding), F.normalize(self.weight))  # Cosine similarity
        cos_theta = cos_theta.clamp(-1.0, 1.0)  # Clamping to prevent overflow
        theta = torch.acos(cos_theta)  # Inverse cosine to get angle
        target_logit = torch.cos(theta + self.m)  # Adding margin
        
        if self.easy_margin:
            target_logit = cos_theta + self.m
        
        # One-hot encoding labels
        one_hot = torch.zeros(cos_theta.size(0), cos_theta.size(1), device=cos_theta.device)
        one_hot.scatter_(1, label.view(-1, 1), 1)
        
        # Loss computation
        loss = F.cross_entropy(target_logit * self.s, one_hot)
        return loss


In [ ]:
# Tải mô hình ArcFace đã huấn luyện sẵn
model = get_model('arcface_r100_v1')
model.to(device)

# Tải bộ dữ liệu của bạn (có nhãn rõ ràng)
train_dataset = get_dataset("your_dataset_path")
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Định nghĩa ArcFace Loss
arcface_loss = ArcFaceLoss()

# Định nghĩa Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

# Fine-tuning trên bộ dữ liệu
for epoch in range(10):  # Số epoch có thể thay đổi
    model.train()
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Tiến hành fine-tuning
        optimizer.zero_grad()
        embeddings = model(images)  # Trích xuất nhúng từ mô hình
        loss = arcface_loss(embeddings, labels)  # Tính loss ArcFace
        loss.backward()
        optimizer.step()
        
        if i % 10 == 0:
            print(f"Epoch [{epoch+1}/10], Step [{i+1}/{len(train_loader)}], Loss: {loss.item():.4f}")
